In [1]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import random
import xgboost as xgb
from sklearn import preprocessing, model_selection
from sklearn.metrics import log_loss
import string
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import  CountVectorizer
from scipy.stats import boxcox
from scipy import stats
from scipy import sparse

/Users/jgzuke/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def fpreproc_leaky(dtrain, dtest, param):
    train, test = dtrain.data, dtest.data
    ntrain = train.shape[0]
    train_test = pd.concat((train, test), axis=0).reset_index(drop=True)
    y_train = dtrain.labels
    
    '''
    # add manager worths
    manager_ids = np.unique(train.manager_id)
    manager_worths = {}
    for manager_id in manager_ids:
        interests = y_train[train.manager_id == manager_id].apply(lambda x: 2 - x)
        if len(interests) > 5:
            manager_worths[manager_id] = sum(interests) / len(interests)
    average_interest = y_train.apply(lambda x: 2 - x)
    average_manager_worth = 0 #sum(average_interest) / len(average_interest)
    train_test['manager_worths'] = train_test['manager_id'].apply(lambda x: manager_worths[x] if x in manager_worths else average_manager_worth)

    
    # add building worths
    building_ids = np.unique(train.building_id)
    building_worths = {}
    for building_id in building_ids:
        interests = y_train[train.building_id == building_id].apply(lambda x: 2 - x)
        if len(interests) > 5:
            building_worths[building_id] = sum(interests) / len(interests)
    average_interest = y_train.apply(lambda x: 2 - x)
    average_building_worth = 0 #sum(average_interest) / len(average_interest)
    train_test['building_worths'] = train_test['building_id'].apply(lambda x: building_worths[x] if x in building_worths else average_building_worth)

    
    # add price by area
    lat_long_price = train_test[['latitude', 'longitude', 'price_per_bedroom']]
    remove_outliers = (np.abs(stats.zscore(lat_long_price)) < 0.15).all(axis=1)
    lat_long_price = lat_long_price[remove_outliers]
    lat_max, lat_min = max(lat_long_price.latitude), min(lat_long_price.latitude)
    long_max, long_min = max(lat_long_price.longitude), min(lat_long_price.longitude)
    lat_scale, long_scale = lat_max - lat_min, long_max - long_min
    costs = np.zeros((100,100))
    num_listings = np.zeros((100,100))
    for lat, long, price_per_bedroom in lat_long_price.values:
        scaled_lat, scaled_long = int((lat - lat_min) * 99 / lat_scale), int((long - long_min) * 99 / long_scale)
        costs[scaled_lat][scaled_long] += price_per_bedroom
        num_listings[scaled_lat][scaled_long] += 1

    price_by_area = []
    for lat, long, price_per_bedroom in train_test[['latitude', 'longitude', 'price_per_bedroom']].values:
        scaled_lat, scaled_long = int((lat - lat_min) * 99 / lat_scale), int((long - long_min) * 99 / long_scale)
        if scaled_lat < 0 or scaled_lat >= 100 or scaled_long < 0 or scaled_long >= 100:
            price_by_area.append(1)
        elif num_listings[scaled_lat][scaled_long] > 8:
            price_by_area.append(price_per_bedroom / (costs[scaled_lat][scaled_long] / num_listings[scaled_lat][scaled_long]))
        else:
            cost = 0
            num = 0
            for i in range(scaled_lat - 1, scaled_lat + 2):
                for j in range(scaled_long - 1, scaled_long + 2):
                    if i > 0 and i < 100 and j >= 0 and j < 100:
                        cost += costs[i][j]
                        num += num_listings[i][j]
            if num > 8:
                price_by_area.append(price_per_bedroom / (cost / num))
            else:
                price_by_area.append(1)

    train_test['price_by_area'] = price_by_area
    
    
    # try adding real - predicted price
    # Try to predict price for a listing and add real_price - expected_price as a feature
    features_to_use = ['BoroCode', 'NTACode', 'bathrooms', 'bedrooms', 'address1', 'other_address', 'building_worths', 'manager_worths']
    feature_to_predict = 'price'
    params = {
        'objective': 'reg:linear',
        'booster':'gblinear',
        'lambda': 0,
        'lambda_bias' : 0,
        'alpha': 0.2
    }
    prices = train_test[feature_to_predict]
    remove_outliers = np.abs(prices-prices.mean())<=(3*prices.std())
    dtrain = xgb.DMatrix(data=train_test[remove_outliers][features_to_use], label=train_test[remove_outliers][feature_to_predict])

    bst = xgb.cv(params, dtrain, 10000, 4, early_stopping_rounds=50, verbose_eval=200)
    best_rounds = np.argmin(bst['test-rmse-mean'])
    print (bst['test-rmse-mean'][best_rounds])
    bst = xgb.train(params, dtrain, best_rounds)
    dtrain = xgb.DMatrix(data=train_test[features_to_use])
    expected_price = bst.predict(dtrain)
    train_test['real_minus_expected_price'] = train_test[feature_to_predict] - expected_price
    train_test['real_over_expected_price'] = train_test[feature_to_predict] / expected_price
    
    
    # enumerated streets / price / price per street (/ num bedrooms) / price / price per street (/ num bedrooms + 0.1*bath)
    reasonable_prices = train_test[np.abs(prices-prices.mean())<=(3*prices.std())]
    for split in []:# ['address1', 'BoroCode', 'NTACode']:
        unique_labels = np.unique(reasonable_prices[split])
        label_to_price = {}
        for label in unique_labels:
            listings = reasonable_prices[reasonable_prices[split] == label]
            if len(listings) > 10:
                label_to_price[label] = sum(listings.price_per_bedroom) / len(listings)
                
        train_test['price_by_{}'.format(split)] = train_test.apply(lambda x: (x.price_per_bedroom / label_to_price[x[split]]) if x[split] in label_to_price else 1, axis=1)

    '''
    
    # remove extra
    train_test.drop('price', axis=1, inplace=True)
    
    dtrain = xgb.DMatrix(data=train_test[:ntrain], label=y_train)
    dtest = xgb.DMatrix(data=train_test[ntrain:], label=dtest.labels)
    return dtrain, dtest, param

In [3]:
current_filter = ['bc_price', 'price_per_bedroom', 'price_per_bathroom', 'price_per_bed_and_bath', 'BoroCode', 'NTACode', 'price', 'latitude', 'price_by_area', 'longitude', 'manager_worths', 'address1', 'real_over_expected_price', 'desc_words_length', 'real_minus_expected_price', 'manager_id', 'Yday', 'building_id', 'building_worths', 'desc_letters_count', 'Day', 'desc_words_count', 'hour', 'bedrooms', 'photos_count', 'features_count', 'Wday', 'sparse_nofee', 'bathrooms', 'sparse_furnished', 'sparse_laundryinbuilding', 'sparse_hardwoodfloors', 'sparse_laundryinunit', 'Month', 'sparse_catsallowed', 'sparse_exclusive', 'street', 'sparse_elevator', 'sparse_prewar', 'sparse_dogsallowed', 'sparse_reducedfee', 'avenue', 'east', 'sparse_doorman', 'sparse_dishwasher', 'other_address', 'sparse_fitnesscenter', 'sparse_privateoutdoorspace', 'sparse_commonoutdoorspace', 'sparse_outdoorspace', 'sparse_loft', 'sparse_diningroom', 'sparse_balcony', 'sparse_highspeedinternet', 'sparse_parkingspace', 'sparse_terrace', 'sparse_swimmingpool', 'west', 'sparse_roofdeck', 'sparse_actualapt', 'sparse_wheelchairaccess', 'sparse_newconstruction', 'Zero_building_id', 'sparse_simplex', 'sparse_patio', 'sparse_garden', 'sparse_multilevel', 'sparse_hardwood', 'sparse_shorttermallowed', 'south', 'sparse_stainlesssteelappliances', 'sparse_fireplace', 'sparse_highceiling', 'sparse_renovated', 'sparse_liveinsuper', 'sparse_storage', 'sparse_garage', 'sparse_dryerinunit', 'sparse_outdoorareas', 'sparse_petsok', 'sparse_lndrybldg', 'sparse_concierge', 'sparse_new', 'sparse_highceilings', 'sparse_onsitelaundry', 'sparse_centrala', 'sparse_flex3', 'sparse_photos', 'sparse_view', 'sparse_publicoutdoor', 'sparse_allutilitiesincluded', 'sparse_residentslounge', 'sparse_newlyrenovated', 'sparse_washerinunit', 'sparse_onsitegarage', 'sparse_assignedparkingspace', 'north', 'sparse_washer', 'sparse_light', 'sparse_dryer', 'sparse_lowrise', 'sparse_sublet', 'sparse_granitekitchen', 'sparse_elev', 'sparse_virtualdoorman', 'sparse_sundeck', 'sparse_rooftopdeck', 'sparse_wallsofwindows', 'sparse_sharesok', 'sparse_duplex', 'sparse_nopets', 'sparse_cable', 'sparse_microwave', 'sparse_wifiaccess', 'sparse_walkincloset', 'sparse_petsonapproval', 'sparse_pool', 'sparse_eatinkitchen', 'sparse_marblebath', 'sparse_live', 'sparse_sauna', 'sparse_greenbuilding', 'sparse_exposedbrick', 'sparse_largelivingroom', 'sparse_bikeroom', 'sparse_highrise', 'sparse_laundry', 'sparse_privateroofdeck', 'sparse_laundryroom', 'sparse_commonbackyard', 'sparse_privatebackyard', 'sparse_parking', 'sparse_privateparking', 'sparse_childrensplayroom', 'sparse_privatebalcony', 'sparse_indoorpool']
leaky = ['manager_worths', 'building_worths', 'price_by_area', 'real_minus_expected_price', 'real_over_expected_price', 'address1']
excluded = ['BoroCode', 'NTACode'] #, 'price_per_bedroom', 'price_per_bathroom', 'price_per_bed_and_bath']
def filter_data_to_columns(data):
    return data[[col for col in current_filter if col not in leaky and col not in excluded]]

# Run CV

In [4]:
%%time
class FakeDMatrix:
    def __init__(self, data, labels=None):
        self.data = data
        self.labels = labels
        self.num = len(data)

    def num_row(self):
        return self.num

    def slice(self, rindex):
        indices = np.zeros(self.num, dtype=np.bool)
        for index in rindex:
            indices[index] = True
        return FakeDMatrix(data=self.data[indices], labels=self.labels[indices])
    
SEED = 777
NFOLDS = 3
y_map = {'low': 2, 'medium': 1, 'high': 0}

params = {
    'eta':.01,
    'colsample_bytree':.8,
    'subsample':.8,
    'seed':0,
    'nthread':16,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'num_class':3,
    'silent':1
}

processed_train = filter_data_to_columns(pd.read_json('data/train.json'))
y_train = pd.read_json('data/train_interest.json')

dtrain = FakeDMatrix(data=processed_train, labels=y_train)
processed_train = None
y_train = None

bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=200, fpreproc=fpreproc_leaky)
best_rounds = np.argmin(bst['test-mlogloss-mean'])
print (bst['test-mlogloss-mean'][best_rounds])
print (best_rounds)

[0]	train-mlogloss:1.09199+5.04997e-05	test-mlogloss:1.09211+2.75802e-05
[200]	train-mlogloss:0.635882+0.00209481	test-mlogloss:0.656265+0.00441317
[400]	train-mlogloss:0.56255+0.00232078	test-mlogloss:0.600346+0.0055522
[600]	train-mlogloss:0.52836+0.00236048	test-mlogloss:0.582248+0.00567578
[800]	train-mlogloss:0.503882+0.00243597	test-mlogloss:0.572789+0.00578039
[1000]	train-mlogloss:0.483854+0.0024353	test-mlogloss:0.566757+0.00574324
[1200]	train-mlogloss:0.466266+0.00218088	test-mlogloss:0.562407+0.00587802
[1400]	train-mlogloss:0.450232+0.00176134	test-mlogloss:0.559268+0.00598037
[1600]	train-mlogloss:0.435421+0.00174083	test-mlogloss:0.556837+0.00598201
[1800]	train-mlogloss:0.421676+0.00157247	test-mlogloss:0.554981+0.00605523
[2000]	train-mlogloss:0.408641+0.00152666	test-mlogloss:0.553541+0.00605184
[2200]	train-mlogloss:0.396407+0.00131219	test-mlogloss:0.552398+0.0060881
[2400]	train-mlogloss:0.384558+0.00128334	test-mlogloss:0.551395+0.0060911
[2600]	train-mlogloss:0.3

# Run To Submit

In [5]:
%%time
processed_test = filter_data_to_columns(pd.read_json('data/test.json'))
listing_id = pd.read_json('data/test_ids.json', typ='series').values

dtest = FakeDMatrix(data=processed_test)
processed_test = None

dtrain_final, dtest_final, _ = fpreproc_leaky(dtrain, dtest, None)
dtrain = None
dtest = None

bst = xgb.train(params, dtrain_final, best_rounds)
dtrain_final = None

preds = bst.predict(dtest_final)
# save for column names
#dtest_final = None

preds = pd.DataFrame(preds)
preds.columns = ['high', 'medium', 'low']
preds['listing_id'] = listing_id
preds.to_csv('data/my_preds.csv', index=None)

importance = bst.get_fscore()
feature_importance = [(feature, (importance[feature])) for feature in importance]
print (sorted(feature_importance, key=lambda x: -x[1]))

[('manager_id', 36363), ('latitude', 35680), ('longitude', 33206), ('bc_price', 29800), ('desc_words_length', 27866), ('building_id', 25638), ('Yday', 23643), ('desc_letters_count', 23153), ('price_per_bedroom', 21492), ('price_per_bathroom', 18244), ('desc_words_count', 17357), ('price_per_bed_and_bath', 17146), ('Day', 17000), ('hour', 16170), ('photos_count', 13436), ('features_count', 12331), ('Wday', 9506), ('bedrooms', 5737), ('sparse_nofee', 4416), ('bathrooms', 3068), ('sparse_laundryinbuilding', 2976), ('sparse_hardwoodfloors', 2734), ('sparse_furnished', 2689), ('sparse_laundryinunit', 2200), ('east', 1951), ('sparse_prewar', 1873), ('street', 1801), ('sparse_catsallowed', 1680), ('west', 1628), ('sparse_exclusive', 1603), ('avenue', 1597), ('sparse_dogsallowed', 1529), ('other_address', 1422), ('sparse_dishwasher', 1393), ('sparse_doorman', 1306), ('sparse_reducedfee', 1300), ('sparse_elevator', 1292), ('sparse_outdoorspace', 1245), ('sparse_fitnesscenter', 1196), ('sparse_l

In [6]:
best_preds = pd.read_csv('data/my_best_preds.csv')
truths = np.argmax(best_preds[['high', 'medium', 'low']].values, axis=1)
# should be < 0.5 for a good submission
log_loss(truths, preds[['high', 'medium', 'low']].values)

0.99658869292590713